In [1]:
!pip install transformers datasets scikit-learn --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cud

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load entire data (assumes enough RAM in Colab)
df = pd.read_csv("/content/drive/MyDrive/test_model_data.csv")

# Combine all text fields
df["text"] = df[["html_title", "h1", "h2", "p"]].fillna("").agg(" ".join, axis=1)

# Prepare labels
text_col = "text"
label_cols = [col for col in df.columns if col not in ["html_title", "h1", "h2", "p", "text"]]

# Convert to HF format
from datasets import Dataset
dataset = Dataset.from_pandas(df[[text_col] + label_cols])


In [ ]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example[text_col], truncation=True, padding="max_length", max_length=256)

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.rename_column(text_col, "input_text")
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'] + label_cols)


In [ ]:
from transformers import BertModel, BertPreTrainedModel
from torch import nn
import torch

class BertMultiLabelClassifier(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.classifier(self.dropout(pooled_output))
        return {"logits": logits}


In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

num_labels = len(label_cols)
train_data, val_data = dataset.train_test_split(test_size=0.1).values()

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    save_total_limit=1,
    logging_dir='./logs',
    load_best_model_at_end=True,
    fp16=True
)

from transformers import AutoConfig
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model = BertMultiLabelClassifier.from_pretrained(model_name, config=config, num_labels=num_labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data
)


In [ ]:
trainer.train()


In [ ]:
trainer.save_model("/content/drive/MyDrive/bert_multilabel_model")
tokenizer.save_pretrained("/content/drive/MyDrive/bert_multilabel_model")


In [11]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
import os

csv_path = "/content/drive/MyDrive/test_model_data.csv"
text_cols = ["html_title", "h1", "h2", "p"]  # Your text columns
label_cols = ["3D printing service", "ATM", "ATV dealer", "ATV rental service", "ATV repair shop", "Aadhaar center", "Aboriginal art gallery", "Abortion clinic", "Abrasives supplier", "Academic department", "Accountant", "Accounting", "Accounting firm", "Accounting school", "Accounting software company", "Acoustical consultant", "Acrylic Dealer", "Acrylic store", "Acupuncture clinic", "Acupuncture school", "Acupuncturist", "Acura dealer", "Addiction treatment center", "Adhesives and glue supplier", "Administrative attorney", "Adoption agency", "Adult DVD store", "Adult day care center", "Adult education school", "Adult entertainment club", "Adult entertainment store", "Adventure Sports Center", "Adventure sports", "Adventure sports center", "Advertising Photographer", "Advertising agency", "Advertising service", "Aerial photographer", "Aerial sports center", "Aero dance class", "Aerobics Class", "Aerobics instructor", "Aeromodel shop", "Aeronautical engineer", "Aerospace company", "African goods store", "After school program", "Aged care", "Aggregate supplier", "Agricultural Engineer", "Agricultural Product Wholesaler", "Agricultural Seed Store", "Agricultural association", "Agricultural cooperative", "Agricultural engineer", "Agricultural loan agency", "Agricultural machinery manufacturer", "Agricultural organization", "Agricultural product wholesaler", "Agricultural production", "Agricultural service", "Agriculture", "Agrochemicals Supplier", "Agrochemicals supplier", "Air compressor repair service", "Air compressor supplier", "Air conditioning contractor", "Air conditioning repair service", "Air conditioning store", "Air conditioning system supplier", "Air duct cleaning service", "Air filter supplier", "Air taxi", "Airbrushing service", "Airbrushing supply store", "Aircraft dealer", "Aircraft maintenance company", "Aircraft manufacturer", "Aircraft rental service", "Aircraft supply store", "Airline", "Airline ticket agency", "Airplane", "Airport", "Airport Shuttle Service Provider", "Airport shuttle service", "Airsoft supply store", "Alcohol manufacturer", "Alcoholic beverage wholesaler", "Alcoholism treatment program", "Allergist", "Alternative Medicine Clinic", "Alternative fuel station", "Alternative medicine clinic", "Alternative medicine practitioner", "Alternator supplier", "Aluminium Supplier", "Aluminum frames supplier", "Aluminum supplier", "Aluminum welder", "Aluminum window", "Ambulance service", "American grocery store", "American restaurant", "Amish furniture store", "Ammunition supplier", "Amphitheater", "Amusement center", "Amusement machine supplier", "Amusement park", "Amusement park ride", "Amusement ride supplier", "Anesthesiologist", "Animal control service", "Animal feed store", "Animal hospital", "Animal park", "Animal protection organization", "Animal rescue service", "Animal shelter", "Animals", "Animation Studio", "Animation studio", "Anime club", "Anodizing service", "Antenna service", "Antique furniture restoration service", "Antique furniture store", "Antique store", "Apartment building", "Apartment complex", "Apartment rental agency", "Apartments", "Apostolic church", "Apparel company", "Appliance parts supplier", "Appliance rental service", "Appliance repair service", "Appliance store", "Appliances customer service", "Applied behavior analysis therapist", "Appraiser", "Apprenticeship center", "Aquaculture farm", "Aquarium", "Aquarium shop", "Aquatic centre", "Arboretum", "Arborist service", "Archaeological museum", "Archery range", "Archery store", "Architect", "Architects association", "Architectural and engineering model maker", "Architectural designer", "Architectural salvage store", "Architecture firm", "Architecture school", "Archive", "Arena", "Argentinian restaurant", "Army & navy surplus shop", "Army museum", "Aromatherapy service", "Aromatherapy supply store", "Art", "Art cafe", "Art center", "Art dealer", "Art gallery", "Art museum", "Art restoration service", "Art school", "Art studio", "Art supply store", "Artificial plant supplier", "Artist", "Artistic handicrafts", "Arts organization", "Asbestos testing service", "Ashram", "Asian fusion restaurant", "Asian grocery store", "Asian restaurant", "Asphalt contractor", "Asphalt mixing plant", "Assemblies of God church", "Assembly hall", "Assisted living facility", "Association / Organization", "Astrologer", "Athletic club", "Athletic field", "Attorney", "Attorney referral service", "Auction house", "Audi dealer", "Audio visual consultant", "Audio visual equipment repair service", "Audio visual equipment supplier", "Audiologist", "Audiovisual equipment rental service", "Auditor", "Auditorium", "Authentic Japanese restaurant", "Auto Accessories Store", "Auto accessories wholesaler", "Auto air conditioning service", "Auto auction", "Auto body parts supplier", "Auto body shop", "Auto bodywork mechanic", "Auto broker", "Auto dent removal service", "Auto electrical service", "Auto glass repair service", "Auto glass shop", "Auto insurance agency", "Auto machine shop", "Auto market", "Auto painting", "Auto parts manufacturer", "Auto parts market", "Auto parts store", "Auto radiator repair service", "Auto repair shop", "Auto restoration service", "Auto spring shop", "Auto sunroof shop", "Auto tag agency", "Auto tune up service", "Auto upholsterer", "Auto window tinting service", "Auto wrecker", "Automation company", "Automobile storage facility", "Aviation", "Aviation consultant", "Aviation training institute", "Awning supplier", "Ayurvedic clinic", "Açaí shop", "BBQ area", "BMW dealer", "BPO Company", "BPO Placement Agency", "BPO company", "BPO placement agency", "Baby clothing store", "Baby store", "Back Office", "Back office", "Backpacker Hostel", "Badminton court", "Bag shop", "Bagel shop", "Bail bonds service", "Bait shop", "Bakery", "Bakery and Cake Shop", "Bakery equipment", "Baking supply store", "Ballet school", "Ballet theater", "Balloon artist", "Balloon ride tour agency", "Balloon store", "Ballroom dance instructor", "Band", "Bank", "Banking and finance", "Banking entrance coaching centre", "Bankruptcy attorney", "Bankruptcy service", "Banner store", "Banquet hall", "Baptist church", "Bar", "Bar & grill", "Bar restaurant furniture store", "Bar stool supplier", "Barbecue restaurant", "Barber school", "Barber shop", "Barber supply store", "Bariatric surgeon", "Bark supplier", "Barrel supplier", "Barrister", "Bartending school", "Baseball", "Baseball club", "Baseball field", "Baseball goods store", "Basketball club", "Basketball court", "Basketball court contractor", "Bathroom remodeler", "Bathroom supply store", "Battery Manufacturer", "Battery Store", "Battery Wholesaler", "Battery manufacturer", "Battery store", "Battery wholesaler", "Batting cage center", "Beach cleaning service", "Beach clothing store", "Beach entertainment shop", "Beach pavillion", "Bead store", "Bead wholesaler", "Bearing supplier", "Beautician", "Beauty", "Beauty Parlour", "Beauty Products Wholesaler", "Beauty product supplier", "Beauty products vending machine", "Beauty products wholesaler", "Beauty salon", "Beauty school", "Beauty supply store", "Bed & breakfast", "Bed shop", "Bedding store", "Bedroom furniture store", "Beer distributor", "Beer garden", "Beer hall", "Beer store", "Belt shop", "Betting agency", "Beverage distributor", "Bicycle Shop", "Bicycle club", "Bicycle rack", "Bicycle rental service", "Bicycle repair shop", "Bicycle wholesaler", "Bike wash", "Bikram yoga studio", "Bilingual school", "Billiards supply store", "Bingo hall", "Biochemical supplier", "Biochemistry lab", "Biotechnology company", "Biotechnology engineer", "Bird control service", "Bird shop", "Bird watching area", "Birth center", "Birth certificate service", "Birth control center", "Bistro", "Blacksmith", "Blast cleaning service", "Blinds shop", "Blood bank", "Blood donation center", "Blood testing service", "Blueprint service", "Board game club", "Board of education", "Boarding house", "Boarding school", "Boat Rental Agency", "Boat accessories supplier", "Boat builders", "Boat cleaning service", "Boat club", "Boat cover supplier", "Boat dealer", "Boat detailing service", "Boat ramp", "Boat rental service", "Boat repair shop", "Boat storage facility", "Boat tour agency", "Boat trailer dealer", "Boating instructor", "Bocce ball court", "Body piercing shop", "Body shaping class", "Boiler manufacturer", "Boiler supplier", "Bonsai plant supplier", "Book publisher", "Book store", "Bookbinder", "Bookkeeping service", "Books wholesaler", "Boot camp", "Boot repair shop", "Boot store", "Botanical garden", "Bottle & can redemption center", "Bottled water supplier", "Bouncy castle hire", "Boutique", "Bowling alley", "Box lunch supplier", "Boxing club", "Boxing gym", "Boys' high school", "Boys' hostel", "Brake shop", "Branding agency", "Brazilian restaurant", "Breakfast restaurant", "Brewery", "Brewing supply store", "Brewpub", "Brick manufacturer", "Bricklayer", "Bridal shop", "Bridge", "British restaurant", "Brunch restaurant", "Bubble tea store", "Buddhist temple", "Buffet restaurant", "Buick dealer", "Builder", "Building consultant", "Building design company", "Building designer", "Building equipment hire service", "Building firm", "Building inspector", "Building materials market", "Building materials store", "Building materials supplier", "Building restoration service", "Building society", "Burglar alarm store", "Burrito restaurant", "Bus and coach company", "Bus charter", "Bus company", "Bus stop", "Bus ticket agency", "Bus tour agency", "Business administration service", "Business attorney", "Business banking service", "Business broker", "Business center", "Business development service", "Business management consultant", "Business networking company", "Business park", "Business related", "Business school", "Business to business service", "Butcher shop", "Butcher shop deli", "CBSE School", "CBSE school", "Cabaret club", "Cabin rental agency", "Cabinet maker", "Cabinet store", "Cable company", "Cadillac dealer", "Cafe", "Cafeteria", "Cajun restaurant", "Cake decorating equipment shop", "Cake shop", "Californian restaurant", "Call center", "Calligraphy lesson", "Calvary Chapel church", "Camera repair shop", "Camera store", "Camper shell supplier", "Campground", "Camping cabin", "Camping farm", "Camping store", "Cancer treatment center", "Candle store", "Candy store", "Cannabis club", "Cannabis store", "Canoe & kayak rental service", "Canoe & kayak store", "Canoe & kayak tour agency", "Canoe and kayak club", "Canoeing area", "Capsule hotel", "Car accessories store", "Car alarm supplier", "Car battery store", "Car dealer", "Car detailing service", "Car factory", "Car finance and loan company", "Car inspection station", "Car leasing service", "Car manufacturer", "Car racing track", "Car rental agency", "Car repair and maintenance service", "Car security system installer", "Car sharing location", "Car stereo store", "Car wash", "Cardiologist", "Care services", "Career guidance service", "Caribbean restaurant", "Carpenter", "Carpet Manufacturer", "Carpet cleaning service", "Carpet installer", "Carpet manufacturer", "Carpet store", "Carpet wholesaler", "Carpooling location", "Carport and pergola builder", "Carriage ride service", "Cars", "Cash and carry wholesaler", "Casino", "Casket service", "Castle", "Casual clothing store", "Cat boarding service", "Caterer", "Catering", "Catering food and drink supplier", "Catholic church", "Catholic school", "Cattery", "Cattle farm", "Cattle market", "Ceiling supplier", "Cell phone accessory store", "Cell phone charging station", "Cell phone store", "Cement Manufacturer", "Cement Supplier", "Cement manufacturer", "Cement supplier", "Cemetery", "Central Government Office", "Ceramic manufacturer", "Ceramics wholesaler", "Certification agency", "Certified public accountant", "Chamber of Commerce", "Chapel", "Charcuterie", "Charity", "Charter school", "Chartered accountant", "Chartered surveyor", "Chauffeur service", "Check cashing service", "Cheese manufacturer", "Cheese shop", "Cheesesteak restaurant", "Chemical Engineer", "Chemical Exporter", "Chemical Manufacturer", "Chemical engineering service", "Chemical exporter", "Chemical industry", "Chemical manufacturer", "Chemical plant", "Chemical wholesaler", "Chemistry lab", "Chess instructor", "Chevrolet dealer", "Chicken hatchery", "Chicken restaurant", "Chicken shop", "Chicken wings restaurant", "Child care agency", "Child health care centre", "Child psychiatrist", "Child psychologist", "Childbirth class", "Children Policlinic", "Children hall", "Children's amusement center", "Children's book store", "Children's camp", "Children's clothing store", "Children's club", "Children's farm", "Children's furniture store", "Children's hospital", "Children's museum", "Children's party buffet", "Children's party service", "Children's store", "Children's theater", "Chimney services", "Chimney sweep", "Chinaware store", "Chinese medicine clinic", "Chinese noodle restaurant", "Chinese restaurant", "Chinese takeaway", "Chiropractor", "Chocolate artisan", "Chocolate cafe", "Chocolate factory", "Chocolate shop", "Chophouse restaurant", "Christian book store", "Christian church", "Christian college", "Christmas market", "Christmas store", "Christmas tree farm", "Chrysler dealer", "Church", "Church of Christ", "Church supply store", "Cider bar", "Cider mill", "Cigar shop", "Cinema equipment supplier", "Circus", "City Department of Public Safety", "City Hall", "City administration", "City clerk's office", "City courthouse", "City employment department", "City government office", "City or town hall", "City park", "City tax office", "Civic center", "Civil defense", "Civil engineer", "Civil engineering company", "Civil examinations academy", "Civil law attorney", "Civil police", "Classified ads newspaper publisher", "Cleaners", "Cleaning Services", "Cleaning products supplier", "Cleaning service", "Clergyman", "Clock and watch maker", "Clock repair service", "Closed circuit television", "Clothes and fabric manufacturer", "Clothes and fabric wholesaler", "Clothes market", "Clothing Accessories Store", "Clothing alteration service", "Clothing manufacturer", "Clothing store", "Clothing supplier", "Clothing wholesale market place", "Clothing wholesaler", "Club", "Co-ed school", "Coaching Center", "Coaching center", "Coal supplier", "Cocktail bar", "Coffee machine supplier", "Coffee roasters", "Coffee shop", "Coffee stand", "Coffee store", "Coffee vending machine", "Coffee wholesaler", "Coffin supplier", "Coin dealer", "Coin operated laundry equipment supplier", "Cold storage facility", "Collectibles store", "College", "College of agriculture", "Comedy club", "Comic book store", "Commercial agent", "Commercial cleaning service", "Commercial photographer", "Commercial printer", "Commercial real estate agency", "Commercial real estate inspector", "Commercial refrigeration", "Commercial refrigerator supplier", "Communications tower", "Community center", "Community college", "Community garden", "Community health centre", "Community school", "Company", "Company registry", "Compressed natural gas station", "Computer Hardware Company", "Computer Rental Agency", "Computer Wholesaler", "Computer accessories store", "Computer club", "Computer consultant", "Computer desk store", "Computer hardware manufacturer", "Computer networking service", "Computer rental agency", "Computer repair service", "Computer security service", "Computer service", "Computer software store", "Computer store", "Computer support and services", "Computer training school", "Computer wholesaler", "Computers", "Concert hall", "Concrete contractor", "Concrete factory", "Concrete metal framework supplier", "Concrete product supplier", "Condiments supplier", "Condominium complex", "Condominium rental agency", "Confectionery store", "Confectionery wholesaler", "Conference center", "Conservation department", "Conservatory of music", "Consignment shop", "Construction", "Construction Material Wholesaler", "Construction and maintenance office", "Construction company", "Construction equipment supplier", "Construction machine dealer", "Construction machine rental service", "Construction material wholesaler", "Consultant", "Consumer advice center", "Contact lenses supplier", "Container service", "Container supplier", "Container terminal", "Containers supplier", "Continental restaurant", "Contractor", "Convenience store", "Convention center", "Convention information bureau", "Cookie shop", "Cooking class", "Cooking school", "Cooler Supplier", "Cooling plant", "Copier repair service", "Copper supplier", "Coppersmith", "Copy shop", "Copying supply store", "Copywriting service", "Corporate campus", "Corporate entertainment service", "Corporate gift supplier", "Corporate office", "Cosmetic Products Manufacturer", "Cosmetic dentist", "Cosmetic products manufacturer", "Cosmetic surgeon", "Cosmetics and perfumes supplier", "Cosmetics industry", "Cosmetics store", "Cosmetics wholesaler", "Cost accounting service", "Costume jewelry shop", "Costume rental service", "Costume store", "Cottage", "Cottage rental", "Council", "Counselor", "Countertop contractor", "Countertop store", "Country club", "County court", "County government office", "Courier service", "Court", "Court reporter", "Courthouse", "Coworking space", "Crab house", "Craft store", "Crane dealer", "Crane rental agency", "Crane service", "Craniosacral therapy", "Credit counseling service", "Credit reporting agency", "Credit union", "Cremation Ground", "Cremation service", "Creole restaurant", "Creperie", "Cricket club", "Cricket ground", "Crime victim service", "Criminal justice attorney", "Crisis center", "Crop grower", "Cruise agency", "Cruise line company", "Cruise terminal", "Crushed stone supplier", "Crypto ATM", "Cuban restaurant", "Culinary school", "Cultural center", "Cultural landmark", "Cupcake shop", "Currency exchange service", "Curtain and upholstery cleaning service", "Curtain store", "Curtain supplier and maker", "Custom home builder", "Custom label printer", "Custom t-shirt store", "Custom tailor", "Customs broker", "Customs consultant", "Customs department", "Customs warehouse", "Cutlery store", "DJ service", "DJ supply store", "DS Automobiles dealer", "DVD store", "Dairy", "Dairy Products Supplier", "Dairy farm", "Dairy farm equipment supplier", "Dairy store", "Dairy supplier", "Dance club", "Dance company", "Dance hall", "Dance school", "Dance store", "Dart bar", "Data Entry Service Provider", "Data center", "Data entry service", "Data recovery service", "Database Management Company", "Database management company", "Dating service", "Day care center", "Day spa", "Deaf service", "Debris removal service", "Debt collecting", "Debt collection agency", "Decal supplier", "Deck builder", "Defence company", "Deli", "Delivery Chinese restaurant", "Delivery Restaurant", "Delivery service", "Demolition contractor", "Dental clinic", "Dental hygienist", "Dental implants periodontist", "Dental implants provider", "Dental insurance agency", "Dental laboratory", "Dental radiology", "Dental school", "Dental supply store", "Dentist", "Denture care center", "Department of Public Safety", "Department of Social Services", "Department of Transportation", "Department of education", "Department of finance", "Department of motor vehicles", "Department store", "Dermatologist", "Design", "Design agency", "Design engineer", "Design institute", "Designer Clothing Store", "Designer clothing store", "Desktop publishing service", "Dessert restaurant", "Dessert shop", "Diabetes center", "Diabetes equipment supplier", "Diabetologist", "Diagnostic center", "Dialysis center", "Diamond Merchant", "Diamond buyer", "Diamond dealer", "Diesel engine dealer", "Diesel engine repair service", "Diesel fuel supplier", "Dietitian", "Digital printer", "Digital printing service", "Dim sum restaurant", "Diner", "Dinner theater", "Direct mail advertising", "Dirt supplier", "Disability equipment supplier", "Disability services and support organization", "Disc golf course", "Disco club", "Discount store", "Discount supermarket", "Display stand manufacturer", "Disposable items shop", "Disposable tableware supplier", "Distance Learning Center", "Distance learning center", "Distillery", "Distribution service", "District attorney", "District government office", "District office", "Dive club", "Dive shop", "Diving center", "Diving contractor", "Divorce lawyer", "Divorce service", "Do-it-yourself shop", "Dock builder", "Doctor", "Dodge dealer", "Dog breeder", "Dog cafe", "Dog day care center", "Dog park", "Dog trainer", "Dog walker", "Dollar store", "Domestic abuse treatment center", "Domestic airport", "Donations center", "Donut shop", "Door manufacturer", "Door shop", "Door supplier", "Door warehouse", "Double glazing installer", "Doula", "Drafting Service Center", "Drafting equipment supplier", "Drafting service", "Drainage service", "Drama school", "Drama theater", "Drawing lessons", "Dress and tuxedo rental service", "Dress store", "Dressmaker", "Dried flower shop", "Drilling contractor", "Drilling equipment supplier", "Drinking water fountain", "Driver and vehicle licensing agency", "Driver's license office", "Drivers license training school", "Driveshaft shop", "Driving school", "Drone service", "Drone shop", "Drug Testing Laboratory", "Drug store", "Drug testing service", "Drum school", "Drum store", "Dry Fruit Store", "Dry cleaner", "Dry fruit store", "Dry ice supplier", "Dry wall contractor", "Dry wall supply store", "Dryer vent cleaning service", "Dude ranch", "Dump truck dealer", "Dumpling restaurant", "Dumpster rental service", "Dynamometer supplier", "E - Commerce Agency", "E commerce agency", "E-commerce service", "EFTPOS equipment supplier", "ENT Specialist", "Ear piercing service", "Earth works company", "Eclectic restaurant", "Ecological park", "Economic Consultant", "Economic consultant", "Economic development agency", "Education", "Education Councils & Boards Office", "Education center", "Educational consultant", "Educational institution", "Educational supply store", "Educational testing service", "Egg supplier", "Elder law attorney", "Electric bicycle store", "Electric generator shop", "Electric motor repair shop", "Electric motor scooter dealer", "Electric motor store", "Electric motor vehicle dealer", "Electric motorcycle dealer", "Electric utility company", "Electric vehicle charging station", "Electric vehicle charging station contractor", "Electrical appliance wholesaler", "Electrical engineer", "Electrical equipment manufacturer", "Electrical equipment supplier", "Electrical installation service", "Electrical products wholesaler", "Electrical repair shop", "Electrical substation", "Electrical supply store", "Electrician", "Electricity Board", "Electricity board", "Electrolysis hair removal service", "Electronic engineering service", "Electronic parts supplier", "Electronics", "Electronics Accessories Wholesaler", "Electronics Company", "Electronics Engineer", "Electronics accessories wholesaler", "Electronics company", "Electronics engineer", "Electronics exporter", "Electronics hire shop", "Electronics manufacturer", "Electronics repair shop", "Electronics store", "Electronics vending machine", "Electronics wholesaler", "Elementary school", "Elevator Manufacturer", "Elevator manufacturer", "Elevator service", "Embassy", "Embossing service", "Embroidery service", "Embroidery shop", "Emergency care physician", "Emergency care service", "Emergency dental service", "Emergency locksmith service", "Emergency room", "Emergency services", "Emergency training", "Emergency training school", "Emergency veterinarian service", "Employment agency", "Employment attorney", "Employment center", "Employment consultant", "Endocrinologist", "Endodontist", "Energy equipment and solutions", "Energy supplier", "Engine Rebuilding Service Station", "Engine rebuilding service", "Engineer", "Engineering consultant", "Engineering school", "English language camp", "English language school", "Engraver", "Entertainer", "Entertainment", "Entertainment agency", "Entertainment and recreation", "Envelope supplier", "Environment office", "Environment renewable natural resources", "Environmental Protection Organization", "Environmental attorney", "Environmental consultant", "Environmental engineer", "Environmental health service", "Environmental organization", "Environmental program", "Environmental protection organization", "Episcopal church", "Equestrian facility", "Equestrian store", "Equipment Exporter", "Equipment exporter", "Equipment importer", "Equipment rental agency", "Equipment supplier", "Escape room center", "Escrow service", "Espresso bar", "Establishment", "Estate appraiser", "Estate liquidator", "Estate litigation attorney", "Estate planning attorney", "Ethnographic museum", "European restaurant", "Evangelical church", "Event management company", "Event planner", "Event technology service", "Event ticket seller", "Event venue", "Excavating contractor", "Executive search firm", "Executive suite rental agency", "Executor", "Exercise equipment store", "Exhibit", "Exhibition and trade centre", "Exhibition planner", "Exporter", "Extended stay hotel", "Eye care center", "Eyebrow bar", "Eyelash salon", "FMCG Manufacturer", "FMCG goods wholesaler", "FMCG manufacturer", "Fabric product manufacturer", "Fabric store", "Fabric wholesaler", "Fabrication Engineer", "Fabrication engineer", "Facial spa", "Factory equipment supplier", "Faculty of media and information science", "Faculty of pharmacy", "Faculty of science", "Fair", "Fairground", "Family counselor", "Family day care service", "Family law attorney", "Family planning center", "Family planning counselor", "Family practice physician", "Family restaurant", "Family service center", "Farm", "Farm bureau", "Farm equipment repair service", "Farm equipment supplier", "Farm household tour", "Farm school", "Farm shop", "Farmers' market", "Farmstay", "Fashion accessories store", "Fashion design school", "Fashion designer", "Fast food restaurant", "Fastener supplier", "Fax Service Provider", "Fax service", "Federal credit union", "Federal government office", "Feed manufacturer", "Fence contractor", "Fence supply store", "Feng shui consultant", "Ferry service", "Fertility clinic", "Fertility physician", "Fertilizer Store", "Fertilizer supplier", "Festival", "Festival hall", "Fiat dealer", "Fiber Optic Products Supplier", "Fiber optic products supplier", "Fiberglass repair service", "Fiberglass supplier", "Figurine shop", "Filipino restaurant", "Film Production Company", "Film and photograph library", "Film production company", "Filtration plant", "Finance broker", "Financial advisor", "Financial audit", "Financial consultant", "Financial institution", "Financial planner", "Fine dining restaurant", "Fingerprinting Service Provider", "Fingerprinting service", "Finishing materials supplier", "Fire alarm supplier", "Fire damage restoration service", "Fire department equipment supplier", "Fire fighters academy", "Fire proofing contractor", "Fire protection consultant", "Fire protection equipment supplier", "Fire protection service", "Fire protection system supplier", "Fire station", "Firearms academy", "Fireplace manufacturer", "Fireplace store", "Firewood supplier", "Fireworks store", "Fireworks supplier", "First aid station", "Fish & chips restaurant", "Fish farm", "Fish processing", "Fish restaurant", "Fish store", "Fishing area", "Fishing camp", "Fishing charter", "Fishing club", "Fishing pier", "Fishing pond", "Fishing store", "Fitness center", "Fitness equipment wholesaler", "Fitted furniture supplier", "Fixed-base operator", "Flag store", "Flavours", "Flavours fragrances and aroma supplier", "Flea market", "Flight school", "Floor refinishing service", "Floor sanding and polishing service", "Flooring contractor", "Flooring store", "Floridian restaurant", "Florist", "Flour mill", "Flower delivery", "Flower designer", "Flower market", "Foam rubber supplier", "Food", "Food & Beverage Exporter", "Food Processing Company", "Food and beverage consultant", "Food and beverage exporter", "Food bank", "Food broker", "Food court", "Food machinery supplier", "Food manufacturer", "Food manufacturing supply", "Food processing company", "Food processing equipment", "Food producer", "Food products supplier", "Food seasoning manufacturer", "Food service", "Food store", "Foot bath", "Foot care", "Foot massage parlor", "Football Field", "Football club", "Footwear wholesaler", "Ford dealer", "Foreclosure service", "Foreign Exchange Students Organization", "Foreign Trade Consultant", "Foreign consulate", "Foreign exchange students organization", "Foreign languages program school", "Foreign trade consultant", "Forensic Consultant", "Forensic consultant", "Forestry service", "Forklift dealer", "Forklift rental service", "Formal Clothing Store", "Formal wear store", "Fortress", "Fortune telling services", "Foster care service", "Foundation", "Foundry", "Fountain contractor", "Fragrances & Aroma Supplier", "Fraternal organization", "Free clinic", "Free parking lot", "Freight Forwarding Agency", "Freight forwarding service", "French language school", "French restaurant", "Fresh food market", "Fried chicken takeaway", "Frozen dessert supplier", "Frozen food manufacturer", "Frozen food store", "Frozen yogurt shop", "Fruit and vegetable processing", "Fruit and vegetable store", "Fruit and vegetable wholesaler", "Fruit wholesaler", "Fruits wholesaler", "Fuel supplier", "Full dress rental service", "Function room facility", "Funeral celebrant service", "Funeral director", "Funeral home", "Fur coat shop", "Fur service", "Furnace parts supplier", "Furnace repair service", "Furnace store", "Furnished apartment building", "Furniture Manufacturer", "Furniture accessories", "Furniture accessories supplier", "Furniture maker", "Furniture manufacturer", "Furniture rental service", "Furniture repair shop", "Furniture store", "Furniture wholesaler", "Fusion restaurant", "Futon store", "GMC dealer", "GPS supplier", "Gambling house", "Game store", "Garage builder", "Garage door supplier", "Garbage collection service", "Garbage dump", "Garbage dump service", "Garden", "Garden building supplier", "Garden center", "Garden furniture shop", "Garden machinery supplier", "Gardener", "Garment Exporter", "Garment exporter", "Gas Agency", "Gas company", "Gas cylinders supplier", "Gas engineer", "Gas installation service", "Gas logs supplier", "Gas shop", "Gas station", "Gasfitter", "Gasket manufacturer", "Gastroenterologist", "Gastrointestinal surgeon", "Gastropub", "Gated community", "Gay & lesbian organization", "Gay bar", "Gay night club", "Gazebo builder", "Gemologist", "Gemstone Jeweler", "Genealogist", "General contractor", "General education school", "General hospital", "General practice attorney", "General practitioner", "General store", "Geological research company", "Geological service", "Geotechnical Engineer", "Geotechnical engineer", "Geriatrician", "German language school", "German restaurant", "Gift basket store", "Gift shop", "Gift wrap store", "Girls' high school", "Girls' hostel", "Glass & mirror shop", "Glass block supplier", "Glass blower", "Glass cutting service", "Glass engraving service", "Glass etching service", "Glass industry", "Glass manufacturer", "Glass merchant", "Glass repair service", "Glass shop", "Glasses repair service", "Glassware manufacturer", "Glassware store", "Glassware wholesaler", "Glazier", "Gluten-free restaurant", "Go-kart track", "Goan restaurant", "Gold dealer", "Gold mining company", "Goldsmith", "Golf cart dealer", "Golf club", "Golf course", "Golf course builder", "Golf driving range", "Golf instructor", "Golf shop", "Gospel church", "Gourmet grocery store", "Government", "Government college", "Government economic program", "Government hospital", "Government office", "Government school", "Graduate school", "Graffiti removal service", "Grain elevator", "Granite supplier", "Graphic designer", "Gravel pit", "Gravel plant", "Greek restaurant", "Green energy supplier", "Greengrocer", "Greenhouse", "Greeting card shop", "Grill", "Grill store", "Grocery delivery service", "Grocery store", "Group accommodation", "Group home", "Guest house", "Guitar instructor", "Guitar store", "Gun club", "Gun shop", "Gurudwara", "Gutter cleaning service", "Gutter service", "Gym", "Gymnasium school", "Gymnastics center", "Gymnastics club", "Gynecologist", "Gypsum Product Supplier", "Gypsum product supplier", "Gyro restaurant", "HIV testing center", "HVAC contractor", "Hair extension technician", "Hair extensions supplier", "Hair removal service", "Hair replacement service", "Hair salon", "Hair transplantation clinic", "Hairdresser", "Halal restaurant", "Halfway house", "Hamburger restaurant", "Handbags shop", "Handicapped transportation service", "Handicraft", "Handicraft Exporter", "Handicraft exporter", "Handicraft museum", "Handicrafts Wholesaler", "Handicrafts wholesaler", "Handyman/Handywoman/Handyperson", "Hardware store", "Harley-Davidson dealer", "Hat shop", "Haunted house", "Haute couture fashion house", "Hawaiian restaurant", "Hay supplier", "Head start center", "Health", "Health and beauty shop", "Health consultant", "Health counselor", "Health food restaurant", "Health food store", "Health insurance agency", "Health resort", "Health spa", "Hearing aid repair service", "Hearing aid store", "Heart hospital", "Heating contractor", "Heating equipment supplier", "Heating oil supplier", "Helicopter Charter Rental & Leasing", "Helicopter charter", "Helicopter tour agency", "Heliport", "Helium gas supplier", "Helmet shop", "Hematologist", "Herb shop", "Herbal medicine store", "Herbalist", "Heritage building", "Heritage museum", "Heritage preservation", "High court", "High ropes course", "High school", "Higher education", "Higher secondary school", "Hiking area", "Hiking guide", "Hill Station", "Hill station", "Hindu priest", "Hindu temple", "Hip hop dance class", "Historical landmark", "Historical place", "Historical place museum", "Historical society", "History museum", "Hobby store", "Hockey rink", "Hockey supply store", "Holding company", "Holiday apartment", "Holiday apartment rental", "Holiday home", "Holistic medicine practitioner", "Home Stay", "Home and construction", "Home audio store", "Home automation company", "Home builder", "Home cinema installation", "Home goods store", "Home health care service", "Home help", "Home help service agency", "Home improvement store", "Home inspector", "Home insurance agency", "Home staging service", "Home theater store", "Homeless service", "Homeless shelter", "Homeopath", "Homeopathic pharmacy", "Homeowners' association", "Homestay", "Honda dealer", "Honey farm", "Hookah bar", "Hookah store", "Horse boarding stable", "Horse breeder", "Horse rental service", "Horse riding field", "Horse riding school", "Horse trailer dealer", "Horse trainer", "Horseback riding service", "Hose supplier", "Hospice", "Hospital", "Hospital department", "Hospital equipment and supplies", "Hospitality and tourism school", "Hostel", "Hot dog restaurant", "Hot dog stand", "Hot pot restaurant", "Hot tub repair service", "Hot tub store", "Hot water system supplier", "Hotel", "Hotel Management Institute", "Hotel management school", "Hotel supply store", "House cleaning service", "House clearance service", "House sitter", "House sitter agency", "Houseboat rental service", "Household chemicals supplier", "Household goods wholesaler", "Housing association", "Housing authority", "Housing complex", "Housing cooperative", "Housing development", "Housing society", "Housing utility company", "Human resource consulting", "Human resources", "Hunting and fishing store", "Hunting area", "Hunting club", "Hunting preserve", "Hunting store", "Hydraulic engineer", "Hydraulic equipment supplier", "Hydraulic repair service", "Hydroelectric power plant", "Hydroponics equipment supplier", "Hygiene articles wholesaler", "Hypermarket", "Hypnotherapy service", "Hyundai dealer", "ICSE School", "ICSE school", "Ice cream equipment supplier", "Ice cream shop", "Ice skating rink", "Ice supplier", "Image consultant", "Immigration & naturalization service", "Immigration attorney", "Immigration detention centre", "Immunologist", "Import export company", "Importer", "Incense supplier", "Incineration plant", "Income protection insurance agency", "Income tax help association", "Indian grocery store", "Indian restaurant", "Indian sweets shop", "Indian takeaway", "Indoor cycling", "Indoor golf course", "Indoor lodging", "Indoor playground", "Indoor swimming pool", "Industrial Spares & Products Wholesaler", "Industrial chemicals wholesaler", "Industrial consultant", "Industrial design company", "Industrial door supplier", "Industrial engineer", "Industrial equipment supplier", "Industrial framework supplier", "Industrial gas supplier", "Industrial real estate agency", "Industrial spares and products wholesaler", "Industrial supermarket", "Industrial technical engineers association", "Industrial vacuum equipment supplier", "Industry", "Infectious disease physician", "Infiniti dealer", "Information & Referral Services Agency", "Information and technology services", "Information centers", "Information services", "Inn", "Insolvency service", "Institute of technology", "Instruction", "Instrumentation engineer", "Insulation contractor", "Insulation materials store", "Insulator supplier", "Insurance", "Insurance agency", "Insurance attorney", "Insurance broker", "Insurance company", "Intellectual property registry", "Intensivist", "Interior Decorator", "Interior architect office", "Interior construction contractor", "Interior decoration", "Interior designer", "Interior door shop", "Interior fitting contractor", "Interior plant service", "International airport", "International school", "International trade consultant", "Internet", "Internet cafe", "Internet marketing service", "Internet service provider", "Internet shop", "Internist", "Inverter and UPS manufacturer", "Investment bank", "Investment company", "Investment service", "Invitation printing service", "Irish pub", "Iron & Steel Industry", "Iron & Steel Store", "Iron and steel industry", "Iron and steel store", "Iron steel contractor", "Iron ware dealer", "Iron works", "Irrigation equipment supplier", "Italian grocery store", "Italian restaurant", "Izakaya restaurant", "Jaguar dealer", "Jamaican restaurant", "Janitorial equipment supplier", "Janitorial service", "Japanese grocery store", "Japanese language instructor", "Japanese restaurant", "Japanese steakhouse", "Jazz club", "Jeans shop", "Jeep dealer", "Jeweler", "Jewelry appraiser", "Jewelry buyer", "Jewelry designer", "Jewelry engraver", "Jewelry equipment supplier", "Jewelry exporter", "Jewelry manufacturer", "Jewelry repair service", "Jewelry store", "Juice shop", "Jujitsu school", "Junior college", "Junk dealer", "Junk removal service", "Junkyard", "Justice department", "Karaoke", "Karaoke bar", "Karate club", "Karate school", "Kawasaki motorcycle dealer", "Kebab shop", "Kennel", "Kerosene supplier", "Key duplication service", "Kia dealer", "Kickboxing school", "Kindergarten", "Kindergarten School", "Kiosk", "Kitchen furniture store", "Kitchen remodeler", "Kitchen supply store", "Kite shop", "Knife manufacturer", "Knife store", "Knit shop", "Knitwear manufacturer", "Konkani restaurant", "Korean barbecue restaurant", "Korean grocery store", "Korean restaurant", "Kosher grocery store", "Kosher restaurant", "Kung fu school", "LPG conversion", "Labor relations attorney", "Labor union", "Laboratory", "Laboratory equipment supplier", "Lactation service", "Ladder supplier", "Laminating equipment supplier", "Lamination service", "Lamp repair service", "Lamp shade supplier", "Land Planning Authority", "Land Rover dealer", "Land planning authority", "Land surveying office", "Land surveyor", "Landscape architect", "Landscape designer", "Landscape lighting designer", "Landscaper", "Landscaping supply store", "Language school", "Lapidary", "Laser Cutting Service Provider", "Laser cutting service", "Laser equipment supplier", "Laser hair removal service", "Laser tag center", "Lasik surgeon", "Latin American restaurant", "Laundromat", "Laundry", "Laundry service", "Law College", "Law book store", "Law firm", "Law library", "Law school", "Lawn care service", "Lawn equipment rental service", "Lawn irrigation equipment supplier", "Lawn mower repair service", "Lawn mower store", "Lawn sprinkler system contractor", "Lawyer", "Lawyers association", "Learning center", "Leasing service", "Leather cleaning service", "Leather goods manufacturer", "Leather goods store", "Leather goods supplier", "Leather goods wholesaler", "Leather repair service", "Leather wholesaler", "Lebanese restaurant", "Legal affairs bureau", "Legal aid office", "Legal services", "Lexus dealer", "Library", "License bureau", "Life coach", "Life insurance agency", "Light bulb supplier", "Lighting consultant", "Lighting contractor", "Lighting manufacturer", "Lighting products wholesaler", "Lighting store", "Limousine service", "Lincoln dealer", "Line marking service", "Linens store", "Lingerie store", "Linoleum store", "Liquidator", "Liquor store", "Literacy program", "Live music bar", "Live music venue", "Livestock auction house", "Livestock breeder", "Livestock dealer", "Livestock producer", "Loan agency", "Local government office", "Local history museum", "Local medical services", "Lock Store", "Locks Supplier", "Locks supplier", "Locksmith", "Lodge", "Lodging", "Log cabins", "Log home builder", "Logging contractor", "Logistics", "Logistics service", "Loss adjuster", "Lottery retailer", "Lottery shop", "Lounge", "Love hotel", "Low income housing program", "Luggage repair service", "Luggage storage facility", "Luggage store", "Luggage wholesaler", "Lumber store", "Lunch restaurant", "Lutheran church", "Lymph drainage therapist", "MG dealer", "MRI center", "Machine construction", "Machine knife supplier", "Machine maintenance service", "Machine repair service", "Machine shop", "Machine workshop", "Machinery parts manufacturer", "Machining manufacturer", "Magazine store", "Magic store", "Magician", "Mailbox rental service", "Mailbox supplier", "Mailing machine supplier", "Mailing service", "Make-up artist", "Makerspace", "Mammography service", "Management school", "Mandarin restaurant", "Manufactured home transporter", "Manufacturer", "Map store", "Mapping service", "Marble contractor", "Marble supplier", "Marina", "Marine engineer", "Marine supply store", "Marine surveyor", "Maritime museum", "Market", "Market researcher", "Marketing", "Marketing agency", "Marketing consultant", "Marriage Consultant", "Marriage celebrant", "Marriage license bureau", "Marriage or relationship counselor", "Martial arts club", "Martial arts school", "Masonry contractor", "Masonry supply store", "Massage school", "Massage spa", "Massage supply store", "Massage therapist", "Material handling equipment supplier", "Maternity hospital", "Maternity store", "Mathematics school", "Mattress store", "Mausoleum builder", "Mazda dealer", "Measuring instruments supplier", "Meat dish restaurant", "Meat packer", "Meat processor", "Meat products store", "Meat wholesaler", "Mechanic", "Mechanical contractor", "Mechanical engineer", "Mechanical plant", "Media Consultant", "Media company", "Media consultant", "Media house", "Mediation service", "Medical Center", "Medical Equipment Manufacturer", "Medical billing service", "Medical book store", "Medical certificate service", "Medical clinic", "Medical diagnostic imaging center", "Medical equipment manufacturer", "Medical equipment supplier", "Medical examiner", "Medical group", "Medical laboratory", "Medical lawyer", "Medical office", "Medical school", "Medical spa", "Medical supply store", "Medical technology manufacturer", "Medical transcription service", "Medicine exporter", "Meditation Class", "Meditation center", "Meditation instructor", "Mediterranean restaurant", "Meeting planning service", "Meeting room", "Mehndi designer", "Memorial", "Memorial park", "Men's clothing store", "Men's health physician", "Men's tailor", "Mental Hospital", "Mental health clinic", "Mental health service", "Mercantile development", "Mercedes-Benz dealer", "Metal Polisher", "Metal construction company", "Metal detecting equipment supplier", "Metal fabricator", "Metal finisher", "Metal heat treating service", "Metal industry suppliers", "Metal machinery supplier", "Metal polishing service", "Metal processing company", "Metal stamping service", "Metal supplier", "Metal working shop", "Metal workshop", "Metallurgy company", "Metalware dealer", "Metalware producer", "Metaphysical supply store", "Methodist church", "Mexican goods store", "Mexican grocery store", "Mexican restaurant", "Mexican torta restaurant", "Microwave oven repair service", "Middle Eastern restaurant", "Middle school", "Midwife", "Military base", "Military hospital", "Military recruiting office", "Military residence", "Military school", "Milk delivery service", "Mill", "Millwork shop", "Mine", "Mineral supplier", "Mineral water company", "Mineral water wholesaler", "Miniature golf course", "Miniatures store", "Minibus taxi service", "Mining company", "Mining consultant", "Mining engineer", "Mining equipment", "Mirror shop", "Mission", "Mitsubishi dealer", "Mobile caterer", "Mobile hairdresser", "Mobile home dealer", "Mobile home park", "Mobile home rental agency", "Mobile home supply store", "Mobile money agent", "Mobile network operator", "Mobile phone repair shop", "Mobility equipment supplier", "Model design company", "Model portfolio studio", "Model shop", "Modeling agency", "Modeling school", "Modern European restaurant", "Modern Indian restaurant", "Modern art museum", "Modular home builder", "Modular home dealer", "Mold maker", "Molding supplier", "Monastery", "Money order service", "Money transfer service", "Monogramming service", "Montessori school", "Monument", "Monument maker", "Moped dealer", "Mortgage broker", "Mortgage lender", "Mortuary", "Mosque", "Motel", "Motor scooter dealer", "Motor scooter repair shop", "Motor vehicle dealer", "Motorcycle dealer", "Motorcycle insurance agency", "Motorcycle parts store", "Motorcycle rental agency", "Motorcycle repair shop", "Motorcycle shop", "Motoring club", "Motorsports store", "Mountain cabin", "Mountaineering class", "Mover", "Movie rental store", "Movie studio", "Movie theater", "Moving and storage service", "Moving company", "Moving supply store", "Muay Thai boxing gym", "Muffler shop", "Mulch supplier", "Multimedia and electronic book publisher", "Municipal Department of Tourism", "Municipal administration office", "Municipal corporation", "Museum", "Music college", "Music conservatory", "Music instructor", "Music management and promotion", "Music producer", "Music publisher", "Music school", "Music store", "Musical Band and Orchestra", "Musical club", "Musical instrument manufacturer", "Musical instrument rental service", "Musical instrument repair shop", "Musical instrument store", "Musician", "Musician and composer", "Nail salon", "Nanotechnology engineering service", "National forest", "National museum", "National park", "Native american goods store", "Natural Stone Exporter", "Natural goods store", "Natural history museum", "Natural stone exporter", "Natural stone supplier", "Natural stone wholesaler", "Nature preserve", "Naturopathic practitioner", "Needlework shop", "Neon sign shop", "Neonatal physician", "Nephrologist", "Neurologist", "Neurosurgeon", "New American restaurant", "News service", "Newspaper advertising department", "Newspaper distribution service", "Newspaper publisher", "Newsstand", "Night club", "Nissan dealer", "Non vegetarian restaurant", "Non-denominational church", "Non-governmental organization", "Non-profit organization", "Noodle shop", "North Indian restaurant", "Northern Italian restaurant", "Notaries association", "Notary public", "Notions store", "Novelty store", "Nuclear power plant", "Nudist park", "Numerologist", "Nurse practitioner", "Nursery school", "Nursing agency", "Nursing home", "Nursing school", "Nut store", "Nutritionist", "Observatory", "Obstetrician-gynecologist", "Occupational health service", "Occupational medical physician", "Occupational safety and health", "Occupational therapist", "Off roading area", "Off-road race track", "Office", "Office accessories wholesaler", "Office equipment rental service", "Office equipment repair service", "Office equipment supplier", "Office furniture store", "Office refurbishment service", "Office services", "Office space rental agency", "Office supply store", "Office supply wholesaler", "Offset Printer", "Offset printer", "Offset printing service", "Oil & natural gas company", "Oil Store", "Oil Wholesaler", "Oil and gas exploration service", "Oil change service", "Oil field equipment supplier", "Oil refinery", "Oil store", "Oil wholesaler", "Oilfield", "Old Age Home", "Olive oil bottling company", "Olive oil cooperative", "Olive oil manufacturer", "Oncologist", "Online Share Trading Center", "Online share trading center", "Open air museum", "Open university", "Opera company", "Opera house", "Ophthalmologist", "Ophthalmology clinic", "Optical instrument repair service", "Optical products manufacturer", "Optical wholesaler", "Optician", "Optometrist", "Oral and maxillofacial surgeon", "Oral surgeon", "Orchard", "Orchestra", "Orchid grower", "Organic drug store", "Organic farm", "Organic food store", "Organic restaurant", "Organic shop", "Oriental rug store", "Orthodontist", "Orthopedic clinic", "Orthopedic shoe store", "Orthopedic surgeon", "Orthotics & prosthetics service", "Osteopath", "Otolaryngologist", "Otolaryngology clinic", "Outboard motor store", "Outdoor activity organiser", "Outdoor clothing and equipment shop", "Outdoor furniture store", "Outdoor movie theater", "Outdoor sports store", "Outdoor swimming pool", "Outerwear store", "Outlet mall", "Outlet store", "Oxygen equipment supplier", "Oyster bar restaurant", "POS terminal supplier", "PVC industry", "PVC windows supplier", "Package locker", "Packaging company", "Packaging machinery", "Packaging supply store", "Pain control clinic", "Pain management physician", "Paint manufacturer", "Paint store", "Paint stripping service", "Paintball center", "Painter", "Painting", "Painting lessons", "Painting studio", "Paintings store", "Pallet supplier", "Pan-Asian restaurant", "Pancake restaurant", "Paper bag supplier", "Paper distributor", "Paper exporter", "Paper mill", "Paper shredding machine supplier", "Paper store", "Paralegal services provider", "Parasailing ride operator", "Park", "Park & ride", "Parking garage", "Parking lot", "Parking lot for bicycles", "Party equipment rental service", "Party planner", "Party store", "Passport Agent", "Passport agent", "Passport office", "Passport photo processor", "Pasta shop", "Pastry shop", "Patent attorney", "Patent office", "Paternity testing service", "Pathologist", "Patients support association", "Patio enclosure supplier", "Patisserie", "Paving contractor", "Paving materials supplier", "Pawn shop", "Payment terminal", "Payroll Service Provider", "Payroll service", "Pediatric cardiologist", "Pediatric clinic", "Pediatric dentist", "Pediatric dermatologist", "Pediatric neurologist", "Pediatric ophthalmologist", "Pediatric orthopedic surgeon", "Pediatric surgeon", "Pediatrician", "Pen store", "Pentecostal church", "Performing arts group", "Performing arts theater", "Perfume Exporter", "Perfume store", "Perinatal center", "Periodontist", "Permanent make-up clinic", "Personal chef service", "Personal concierge service", "Personal injury attorney", "Personal trainer", "Personal watercraft dealer", "Peruvian restaurant", "Pest control service", "Pet adoption service", "Pet boarding service", "Pet cemetery", "Pet funeral service", "Pet groomer", "Pet moving service", "Pet sitter", "Pet store", "Pet supply store", "Pet trainer", "Petrochemical engineering service", "Petroleum Products Company", "Petroleum products company", "Pharmaceutical Products Wholesaler", "Pharmaceutical company", "Pharmaceutical lab", "Pharmaceutical products wholesaler", "Pharmaceuticals Exporter", "Pharmacy", "Pho restaurant", "Phone repair service", "Photo agency", "Photo booth", "Photo lab", "Photo restoration service", "Photo shop", "Photocopiers supplier", "Photographer", "Photography Class", "Photography Institute", "Photography class", "Photography school", "Photography service", "Photography studio", "Physical examination center", "Physical fitness program", "Physical rehabilitation center", "Physical therapist", "Physical therapy clinic", "Physician assistant", "Physician referral service", "Physiotherapist", "Physiotherapy Center", "Physiotherapy equipment supplier", "Piano bar", "Piano instructor", "Piano moving service", "Piano repair service", "Piano store", "Piano tuning service", "Pick your own farm produce", "Pickleball court", "Picnic ground", "Picture frame shop", "Pie shop", "Pig farm", "Pilates studio", "Pile driving service", "Pilgrimage place", "Pinball machine supplier", "Pipe Manufacturer", "Pipe manufacturer", "Pipe supplier", "Pipes Wholesaler", "Pipes wholesaler", "Pizza Takeout", "Pizza delivery", "Pizza restaurant", "Place of worship", "Planetarium", "Plant and machinery hire", "Plant nursery", "Plasterer", "Plastic Injection Molding Workshop", "Plastic Products Supplier", "Plastic Wholesaler", "Plastic bag supplier", "Plastic bags wholesaler", "Plastic fabrication company", "Plastic injection molding service", "Plastic products supplier", "Plastic products wholesaler", "Plastic resin manufacturer", "Plastic surgeon", "Plastic surgery clinic", "Plating service", "Playground", "Playground equipment supplier", "Playgroup", "Plaza", "Plumber", "Plumbing supply store", "Plus size clothing store", "Plywood supplier", "Pneumatic tools supplier", "Podiatrist", "Poke bar", "Police", "Police academy", "Police department", "Police station", "Police supply store", "Political party office", "Polygraph service", "Polymer supplier", "Polytechnic College", "Polytechnic institute", "Polythene and plastic sheeting supplier", "Pond contractor", "Pond fish supplier", "Pond supply store", "Pony ride service", "Pool billard club", "Pool cleaning service", "Pool hall", "Popcorn store", "Porsche dealer", "Port authority", "Port operating company", "Portable building manufacturer", "Portable toilet supplier", "Portrait studio", "Portuguese restaurant", "Post office", "Poster store", "Pottery classes", "Pottery manufacturer", "Pottery store", "Poultry farm", "Poultry store", "Powder coating service", "Power plant consultant", "Power plant equipment supplier", "Power station", "Powersports vehicle dealer", "Practitioner service location", "Precision engineer", "Prefabricated house companies", "Pregnancy care center", "Prepaid SIM card store", "Preparatory school", "Presbyterian church", "Preschool", "Press advisory", "Pressure washing service", "Priest", "Primary school", "Print shop", "Printer ink refill store", "Printer repair service", "Printing equipment and supplies", "Printing equipment supplier", "Prison", "Private College", "Private Tutor", "Private college", "Private educational institution", "Private equity firm", "Private golf course", "Private hospital", "Private investigator", "Private sector bank", "Private tutor", "Private university", "Probation office", "Process server", "Produce market", "Produce wholesaler", "Professional and hobby associations", "Professional association", "Professional organizer", "Professional services", "Promotional products supplier", "Propane supplier", "Propeller shop", "Property administration service", "Property investment company", "Property maintenance", "Property management company", "Prosthodontist", "Protective clothing supplier", "Protestant church", "Psychiatric hospital", "Psychiatrist", "Psychic", "Psychoanalyst", "Psychologist", "Psychoneurological specialized clinic", "Psychotherapist", "Pub", "Public bathroom", "Public beach", "Public defender's office", "Public educational institution", "Public golf course", "Public health department", "Public library", "Public parking space", "Public relations firm", "Public safety office", "Public sector bank", "Public swimming pool", "Public transportation system", "Public university", "Public utility", "Public works department", "Publisher", "Pulmonologist", "Pump supplier", "Pumpkin patch", "Punjabi restaurant", "Quantity surveyor", "Quarry", "Quilt shop", "RV dealer", "RV detailing service", "RV inspection service", "RV park", "RV repair shop", "RV storage facility", "RV supply store", "Race car dealer", "Racecourse", "Racing car parts store", "Radiator repair service", "Radiator shop", "Radio broadcaster", "Radiologist", "Raft trip outfitter", "Rafting", "Rail museum", "Railing contractor", "Railroad company", "Railroad contractor", "Railroad equipment supplier", "Railroad ties supplier", "Railway Ticket Agent", "Railway services", "Rainwater tank supplier", "Ram dealer", "Ramen restaurant", "Ranch", "Rare book store", "Ready mix concrete supplier", "Real Estate Builders & Construction Company", "Real estate", "Real estate agency", "Real estate agent", "Real estate appraiser", "Real estate attorney", "Real estate auctioneer", "Real estate consultant", "Real estate developer", "Real estate rental agency", "Real estate school", "Real estate surveyor", "Reclamation centre", "Record company", "Record store", "Recording studio", "Records storage facility", "Recreation", "Recreation center", "Recreational vehicle rental agency", "Recruiter", "Recycling center", "Recycling drop-off location", "Reflexologist", "Refrigerated transport service", "Refrigerator repair service", "Refrigerator store", "Regional airport", "Regional government office", "Registered general nurse", "Registration office", "Registry office", "Rehabilitation center", "Rehearsal studio", "Reiki therapist", "Religious book store", "Religious destination", "Religious goods store", "Religious institution", "Religious organization", "Religious school", "Religious statue manufacturer", "Remodeler", "Rental", "Renter's insurance agency", "Repair service", "Repairs", "Reproductive health clinic", "Reptile store", "Rescue squad", "Research Engineer", "Research and product development", "Research engineer", "Research foundation", "Research institute", "Residential college", "Residents association", "Resort hotel", "Rest stop", "Restaurant", "Restaurant supply store", "Resume service", "Retail space rental agency", "Retaining wall supplier", "Retirement community", "Retirement home", "Retreat center", "Rheumatologist", "Rice mill", "Rice restaurant", "Rice wholesaler", "Road Contractor", "Road construction company", "Road construction machine repair service", "Road cycling", "Road safety town", "Rock climbing", "Rock climbing gym", "Rock climbing instructor", "Rock shop", "Rodeo", "Rolled metal products supplier", "Roller coaster", "Roller skating club", "Roller skating rink", "Roofing contractor", "Roofing supply store", "Rubber products supplier", "Rubber stamp store", "Rug store", "Running store", "Rustic furniture store", "SCUBA instructor", "SCUBA tour agency", "STD clinic", "STD testing service", "Saddlery", "Safe & vault shop", "Safety equipment supplier", "Sailing club", "Sailing event area", "Sailing school", "Salad shop", "Salsa classes", "Salvage dealer", "Salvage yard", "Sand & gravel supplier", "Sand plant", "Sandblasting service", "Sandwich shop", "Sanitary inspection", "Sanitation service", "Saree Shop", "Satellite Communication Services Provider", "Satellite communication service", "Sauna", "Sauna store", "Savings bank", "Saw mill", "Saw sharpening service", "Scaffolding rental service", "Scaffolding service", "Scale model club", "Scale repair service", "Scale supplier", "Scenic spot", "Scenography company", "School", "School administration office", "School bus service", "School center", "School district office", "School for the deaf", "School for the visually impaired", "School supply store", "Science museum", "Scientific equipment supplier", "Scooter rental service", "Scooter repair shop", "Scouting", "Scrap metal dealer", "Scrapbooking store", "Screen printer", "Screen printing shop", "Screen printing supply store", "Screen repair service", "Screen store", "Screw supplier", "Sculptor", "Sculpture", "Sculpture museum", "Seafood market", "Seafood restaurant", "Seafood wholesaler", "Seal shop", "Seasonal goods store", "Seat dealer", "Second hand store", "Secondary school", "Security guard service", "Security service", "Security system installation service", "Security system supplier", "Seed supplier", "Self defense school", "Self service car wash", "Self service restaurant", "Self-catering accommodation", "Self-storage facility", "Semi conductor supplier", "Seminary", "Senior citizen center", "Senior high school", "Septic system service", "Service Apartment", "Service establishment", "Serviced accommodation", "Serviced apartment", "Services", "Seventh-day Adventist church", "Sewage disposal service", "Sewage treatment plant", "Sewing company", "Sewing machine repair service", "Sewing machine store", "Sewing shop", "Sexologist", "Shabu-shabu restaurant", "Shared-use commercial kitchen", "Sharpening service", "Shed builder", "Sheet metal contractor", "Sheet music store", "Shelving store", "Sheriff's department", "Shipbuilding and repair company", "Shipping and mailing", "Shipping and mailing service", "Shipping company", "Shipping equipment industry", "Shipping service", "Shipyard", "Shoe factory", "Shoe repair shop", "Shoe store", "Shooting event area", "Shooting range", "Shop supermarket furniture store", "Shopfitter", "Shopping mall", "Short term apartment rental agency", "Shower door shop", "Showroom", "Shredding Services Provider", "Shredding service", "Sicilian restaurant", "Siding contractor", "Sightseeing Tour Operator", "Sightseeing tour agency", "Sign shop", "Silk plant shop", "Silk store", "Silversmith", "Skate sharpening service", "Skate shop", "Skateboard park", "Skateboard shop", "Ski club", "Ski rental service", "Ski repair service", "Ski resort", "Ski school", "Ski shop", "Skiing Club", "Skin care clinic", "Skin care products vending machine", "Skydiving center", "Skylight contractor", "Slaughterhouse", "Sleep clinic", "Small Claims Service Agency", "Small appliance repair service", "Small claims assistance service", "Small engine repair service", "Small plates restaurant", "Smart dealer", "Smart shop", "Smog inspection station", "Smoke shop", "Snack bar", "Snow removal service", "Snowboard rental service", "Snowboard shop", "Snowmobile dealer", "Snowmobile rental service", "Soapland", "Soccer club", "Soccer field", "Soccer practice", "Soccer store", "Social club", "Social security attorney", "Social security financial department", "Social security office", "Social services organization", "Social welfare center", "Social worker", "Sod supplier", "Sofa store", "Soft drinks shop", "Softball club", "Softball field", "Software Training Institute", "Software company", "Software training institute", "Soil testing service", "Solar Energy Company", "Solar energy company", "Solar energy equipment supplier", "Solar energy system service", "Solar hot water system supplier", "Solar panel maintenance service", "Solar photovoltaic power plant", "Solid waste engineer", "Soul food restaurant", "Soup restaurant", "Soup shop", "South Asian restaurant", "South Indian restaurant", "Southeast Asian restaurant", "Southern restaurant (US)", "Southwestern restaurant (US)", "Souvenir manufacturer", "Souvenir store", "Spa", "Spa and health club", "Spanish restaurant", "Special education school", "Special educator", "Specialized clinic", "Specialized hospital", "Speech & Hearing Specialist", "Speech pathologist", "Spice exporter", "Spice store", "Spice wholesaler", "Spices Exporter", "Spices Wholesaler", "Spiritist center", "Sport tour agency", "Sporting goods store", "Sports accessories wholesaler", "Sports activity location", "Sports association", "Sports bar", "Sports card store", "Sports club", "Sports complex", "Sports equipment rental service", "Sports massage therapist", "Sports medicine clinic", "Sports medicine physician", "Sports memorabilia store", "Sports nutrition store", "Sports school", "Sportswear store", "Sportwear manufacturer", "Spring supplier", "Stable", "Stadium", "Stage", "Stage lighting equipment supplier", "Stained glass studio", "Stainless steel plant", "Stair contractor", "Stall installation service", "Stamp shop", "Staple food package", "State Board School", "State Department Science Technology", "State Department of Transportation", "State employment department", "State government office", "State park", "State police", "Stationery manufacturer", "Stationery store", "Stationery wholesaler", "Statuary", "Steak house", "Steel construction company", "Steel distributor", "Steel drum supplier", "Steel erector", "Steel fabricator", "Steel framework contractor", "Steelwork design service", "Stereo repair service", "Sticker Manufacturer", "Sticker manufacturer", "Stock broker", "Stock exchange building", "Stone carving", "Stone cutter", "Stone supplier", "Storage", "Storage facility", "Store", "Store equipment supplier", "Stores and shopping", "Structural engineer", "Stucco contractor", "Student career counseling office", "Student dormitory", "Student housing center", "Student union", "Students support association", "Study at home school", "Studying center", "Stylist", "Subaru dealer", "Sugar factory", "Summer camp", "Summer camp organizer", "Sunglasses store", "Sunroom contractor", "Superannuation consultant", "Supermarket", "Supplier", "Support group", "Surf school", "Surf shop", "Surgeon", "Surgical center", "Surgical oncologist", "Surgical products wholesaler", "Surgical supply store", "Surplus store", "Surveyor", "Sushi restaurant", "Sushi takeaway", "Suzuki dealer", "Suzuki motorcycle dealer", "Sweet shop", "Sweets and dessert buffet", "Swim club", "Swimming facility", "Swimming instructor", "Swimming lake", "Swimming pool", "Swimming pool contractor", "Swimming pool repair service", "Swimming pool supply store", "Swimwear store", "T-shirt company", "T-shirt store", "TB clinic", "Tack shop", "Taco restaurant", "Taekwondo school", "Tai chi school", "Tailor", "Taiwanese restaurant", "Takeout Restaurant", "Takeout restaurant", "Talent agency", "Tannery", "Tanning salon", "Tapas bar", "Tapas restaurant", "Tattoo and piercing shop", "Tattoo artist", "Tattoo removal service", "Tattoo shop", "Tax", "Tax assessor", "Tax attorney", "Tax collector's office", "Tax consultant", "Tax department", "Tax preparation", "Tax preparation service", "Taxi service", "Taxicab Stand", "Taxidermist", "Tea Exporter", "Tea Manufacturer", "Tea Wholesaler", "Tea exporter", "Tea house", "Tea manufacturer", "Tea market place", "Tea store", "Tea wholesaler", "Teachers college", "Technical school", "Technical university", "Technology museum", "Technology park", "Teeth whitening service", "Telecommunications", "Telecommunications contractor", "Telecommunications engineer", "Telecommunications equipment supplier", "Telecommunications service provider", "Telemarketing service", "Telephone Marketing Agency", "Telephone answering service", "Telephone company", "Telephone exchange", "Television Channel", "Television repair service", "Television station", "Temp agency", "Tennis club", "Tennis court", "Tennis court construction company", "Tennis store", "Tent House Supplier", "Tent rental service", "Teppanyaki restaurant", "Test preparation center", "Tex-Mex restaurant", "Textile Exporter", "Textile engineer", "Textile exporter", "Textile mill", "Thai massage therapist", "Thai restaurant", "Theater company", "Theater production", "Theater supply store", "Theme park", "Thermal Energy Company", "Thermal baths", "Thermal power plant", "Thread supplier", "Thrift store", "Ticket office", "Tiki bar", "Tile cleaning service", "Tile contractor", "Tile manufacturer", "Tile store", "Timeshare agency", "Tire repair shop", "Tire service", "Tire shop", "Title company", "Tobacco shop", "Tobacco supplier", "Toiletries store", "Toner cartridge supplier", "Tool & die shop", "Tool grinding service", "Tool manufacturer", "Tool rental service", "Tool repair shop", "Tool store", "Tool wholesaler", "Toolroom", "Topography company", "Topsoil supplier", "Tour agency", "Tour operator", "Tourism", "Tourism Development Corporation", "Tourism development corporation", "Tourist attraction", "Tourist information center", "Towing equipment provider", "Towing service", "Townhouse complex", "Toy and game manufacturer", "Toy manufacturer", "Toy store", "Toyota dealer", "Tractor dealer", "Tractor repair shop", "Trade fair construction company", "Trade school", "Trading card store", "Traditional American restaurant", "Traditional market", "Traditional restaurant", "Traffic officer", "Trailer dealer", "Trailer manufacturer", "Trailer rental service", "Trailer repair shop", "Trailer supply store", "Train station", "Train ticket agency", "Train ticket office", "Training centre", "Training consultant", "Training provider", "Transcription service", "Transit depot", "Transit station", "Translation service", "Transmission shop", "Transportation escort service", "Transportation infrastructure", "Transportation service", "Travel", "Travel agency", "Travel clinic", "Travel lounge", "Travel services", "Tree farm", "Tree service", "Trial attorney", "Tribal headquarters", "Trophy shop", "Tropical fish store", "Truck accessories store", "Truck dealer", "Truck driving school", "Truck parts supplier", "Truck rental agency", "Truck repair shop", "Truck stop", "Truck topper supplier", "Trucking company", "Truss manufacturer", "Trust bank", "Tune up supplier", "Tuning automobile", "Turf supplier", "Turkish restaurant", "Turnery", "Tutoring service", "Tuxedo shop", "Two Wheeler Dealer", "Two Wheeler Repair Shop", "Typewriter repair service", "Typing service", "Typist", "Tyre manufacturer", "Udon noodle restaurant", "Underwear store", "Unemployment office", "Uniform store", "United Methodist church", "University", "University hospital", "University library", "Upholstery cleaning service", "Upholstery shop", "Urban planning department", "Urgent care center", "Urologist", "Urology clinic", "Used CD store", "Used appliance store", "Used auto parts store", "Used bicycle shop", "Used book store", "Used car dealer", "Used clothing store", "Used computer store", "Used furniture store", "Used game store", "Used motorcycle dealer", "Used musical instrument store", "Used office furniture store", "Used tire shop", "Used truck dealer", "Utility contractor", "Utility trailer dealer", "Vacation home rental agency", "Vacuum cleaner repair shop", "Vacuum cleaner store", "Vacuum cleaning system supplier", "Valet parking service", "Van rental agency", "Vaporizer store", "Variety store", "Vascular surgeon", "Vastu consultant", "Vegan restaurant", "Vegetable wholesale market", "Vegetable wholesaler", "Vegetarian cafe and deli", "Vegetarian restaurant", "Vehicle exporter", "Vehicle inspection service", "Vehicle shipping agent", "Vehicle wrapping service", "Vending machine supplier", "Venezuelan restaurant", "Ventilating equipment manufacturer", "Venture Capital Fund", "Venture capital company", "Veterans affairs department", "Veterans center", "Veterans hospital", "Veterans organization", "Veterinarian", "Veterinary care", "Veterinary pharmacy", "Video Conferencing Service Provider", "Video arcade", "Video camera repair service", "Video conferencing equipment supplier", "Video conferencing service", "Video duplication service", "Video editing service", "Video equipment repair service", "Video game rental service", "Video game rental store", "Video game store", "Video production service", "Video store", "Vietnamese restaurant", "Villa", "Village hall", "Vineyard", "Vintage clothing store", "Vinyl sign shop", "Violin shop", "Virtual office rental", "Visa Agent", "Visa and passport office", "Visa consulting service", "Visitor center", "Vitamin & supplements store", "Vocal instructor", "Vocational school", "Vocational secondary school", "Volkswagen dealer", "Volleyball court", "Volunteer organization", "Volvo dealer", "Voter registration office", "Walk-in clinic", "Wallpaper installer", "Wallpaper store", "War museum", "Warehouse", "Warehouse club", "Warehouse store", "Washer & dryer repair service", "Washer & dryer store", "Waste management service", "Watch manufacturer", "Watch repair service", "Watch store", "Water Filter Supplier", "Water Pump Supplier", "Water Tank Cleaners", "Water Testing Laboratory", "Water cooler supplier", "Water damage restoration service", "Water filter supplier", "Water jet cutting service", "Water park", "Water pump supplier", "Water purification company", "Water ski shop", "Water skiing instructor", "Water softening equipment supplier", "Water sports equipment rental service", "Water tank cleaning service", "Water testing service", "Water treatment plant", "Water treatment supplier", "Water utility company", "Water works", "Water works equipment supplier", "Waterproofing service", "Wax supplier", "Waxing hair removal service", "Weather forecast service", "Web hosting company", "Website designer", "Wedding Portrait Studio", "Wedding bakery", "Wedding buffet", "Wedding chapel", "Wedding dress rental service", "Wedding photographer", "Wedding planner", "Wedding portrait studio", "Wedding service", "Wedding souvenir shop", "Wedding store", "Wedding venue", "Weigh station", "Weighing Scale Supplier", "Weight loss service", "Weightlifting area", "Welder", "Welding gas supplier", "Welding supply store", "Well drilling contractor", "Wellness center", "Wellness hotel", "Wellness program", "Western apparel store", "Whale watching tour agency", "Wheel alignment service", "Wheel store", "Wheelchair rental service", "Wheelchair repair service", "Wheelchair store", "Wholesale bakery", "Wholesale drugstore", "Wholesale florist", "Wholesale food store", "Wholesale grocer", "Wholesale jeweler", "Wholesale market", "Wholesale plant nursery", "Wholesaler", "Wholesaler household appliances", "Wi-Fi spot", "Wig shop", "Wildlife and safari park", "Wildlife park", "Wildlife refuge", "Wildlife rescue service", "Wind farm", "Wind turbine builder", "Window cleaning service", "Window installation service", "Window supplier", "Window tinting service", "Window treatment store", "Wine bar", "Wine cellar", "Wine club", "Wine storage facility", "Wine store", "Wine wholesaler and importer", "Winemaking supply store", "Winery", "Women's clothing store", "Women's college", "Women's health clinic", "Women's organization", "Women's personal trainer", "Women's protection service", "Women's shelter", "Wood and laminate flooring supplier", "Wood floor installation service", "Wood floor refinishing service", "Wood frame supplier", "Wood stove shop", "Wood supplier", "Wood working class", "Woodworker", "Woodworking supply store", "Work clothes store", "Working women's hostel", "Wrestling school", "X-ray equipment supplier", "X-ray lab", "Yacht broker", "Yacht club", "Yamaha motorcycle dealer", "Yarn store", "Yoga Retreat Center", "Yoga instructor", "Yoga retreat center", "Yoga studio", "Youth care service", "Youth center", "Youth clothing store", "Youth club", "Youth group", "Youth hostel", "Youth organization", "Youth social services organization", "Zoo"]

chunk_size = 10000
save_dir = "/content/tokenized_chunks"

os.makedirs(save_dir, exist_ok=True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

# Read and tokenize in chunks
for i, chunk in enumerate(pd.read_csv(csv_path, chunksize=chunk_size)):
    chunk[text_cols] = chunk[text_cols].fillna("")
    chunk["text"] = chunk[text_cols].agg(" ".join, axis=1)
    ds = Dataset.from_pandas(chunk[["text"] + label_cols])
    ds = ds.map(tokenize_function, batched=True)
    ds.set_format(type="torch", columns=["input_ids", "attention_mask"] + label_cols)
    ds.save_to_disk(os.path.join(save_dir, f"chunk_{i}"))


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9418 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9418 [00:00<?, ? examples/s]

In [6]:
from datasets import load_from_disk, concatenate_datasets

chunk_paths = [os.path.join(save_dir, d) for d in sorted(os.listdir(save_dir)) if d.startswith("chunk_")]
datasets = [load_from_disk(path) for path in chunk_paths]

full_dataset = concatenate_datasets(datasets)
full_dataset = full_dataset.shuffle(seed=42)
train_test = full_dataset.train_test_split(test_size=0.2)


In [10]:
# print(df[label_cols].dtypes)


html_title                            object
p                                     object
h1                                    object
h2                                    object
3D printing service                    int64
                                       ...  
Youth group                            int64
Youth hostel                           int64
Youth organization                     int64
Youth social services organization     int64
Zoo                                    int64
Length: 3448, dtype: object


In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

num_labels = len(label_cols)

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels,
    problem_type="multi_label_classification"
)

def custom_collate(batch):
    batch = {k: torch.stack([d[k] for d in batch]) for k in batch[0]}
    batch["labels"] = batch["labels"].float()
    return batch

def format_labels(example):
    example["labels"] = [float(example[col]) for col in label_cols]  # Explicitly use float
    return example

# Apply before passing to Trainer
train_dataset = train_test["train"].map(format_labels)
eval_dataset = train_test["test"].map(format_labels)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,  # Adjust batch size as needed
    per_device_eval_batch_size=8,  # Adjust batch size as needed
    num_train_epochs=3,  # Number of epochs
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=2,
    fp16=True,  # Use mixed precision if supported
    report_to="none"  # Avoid reporting to external services
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=custom_collate
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/183534 [00:00<?, ? examples/s]

ValueError: could not convert string to float: 'msi sign design environmental graphics sign company architectural scale models interior design weekend directionals contractor'

In [3]:
# import pandas as pd

# df = pd.read_csv( "/content/drive/MyDrive/test_model_data.csv",nrows=5)

# # Get all column names (headings)
# column_names = df.columns.tolist()

# # Display the result
# print(column_names)




['html_title', 'p', 'h1', 'h2', '3D printing service', 'ATM', 'ATV dealer', 'ATV rental service', 'ATV repair shop', 'Aadhaar center', 'Aboriginal art gallery', 'Abortion clinic', 'Abrasives supplier', 'Academic department', 'Accountant', 'Accounting', 'Accounting firm', 'Accounting school', 'Accounting software company', 'Acoustical consultant', 'Acrylic Dealer', 'Acrylic store', 'Acupuncture clinic', 'Acupuncture school', 'Acupuncturist', 'Acura dealer', 'Addiction treatment center', 'Adhesives and glue supplier', 'Administrative attorney', 'Adoption agency', 'Adult DVD store', 'Adult day care center', 'Adult education school', 'Adult entertainment club', 'Adult entertainment store', 'Adventure Sports Center', 'Adventure sports', 'Adventure sports center', 'Advertising Photographer', 'Advertising agency', 'Advertising service', 'Aerial photographer', 'Aerial sports center', 'Aero dance class', 'Aerobics Class', 'Aerobics instructor', 'Aeromodel shop', 'Aeronautical engineer', 'Aerosp

In [4]:
# import pandas as pd

# # Load the CSV file
# df = pd.read_csv( "/content/drive/MyDrive/test_model_data.csv",nrows=5)

# # Get all column names (headings)
# column_names = df.columns.tolist()

# # Convert the column names to the desired format (double-quoted strings)
# formatted_column_names = '["' + '", "'.join(column_names) + '"]'

# # Display the result
# print(formatted_column_names)


["html_title", "p", "h1", "h2", "3D printing service", "ATM", "ATV dealer", "ATV rental service", "ATV repair shop", "Aadhaar center", "Aboriginal art gallery", "Abortion clinic", "Abrasives supplier", "Academic department", "Accountant", "Accounting", "Accounting firm", "Accounting school", "Accounting software company", "Acoustical consultant", "Acrylic Dealer", "Acrylic store", "Acupuncture clinic", "Acupuncture school", "Acupuncturist", "Acura dealer", "Addiction treatment center", "Adhesives and glue supplier", "Administrative attorney", "Adoption agency", "Adult DVD store", "Adult day care center", "Adult education school", "Adult entertainment club", "Adult entertainment store", "Adventure Sports Center", "Adventure sports", "Adventure sports center", "Advertising Photographer", "Advertising agency", "Advertising service", "Aerial photographer", "Aerial sports center", "Aero dance class", "Aerobics Class", "Aerobics instructor", "Aeromodel shop", "Aeronautical engineer", "Aerosp